# Bellman-Ford Algorithm

Invented by Richard Bellman and Lester Ford in 1950s, the Bellman-Ford algorithm is used to find the shortest path from a source vertex to all other vertices in a weighted graph. It is slower than Dijkstra's algorithm, but it is more versatile, as it is capable of handling graphs with **negative** edge weights.

Src: [Wikipedia](https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm)

## Algorithm

The algorithm works by relaxing all the edges in the graph `|V| - 1` times, where `|V|` is the number of vertices in the graph. The relaxation process involves checking if the distance to a vertex `v` can be shortened by taking the edge `(u, v)`.

The algorithm also checks for negative cycles in the graph. A negative cycle is a cycle whose total weight is negative. If a negative cycle is found, the algorithm returns `False`, as the shortest path to some vertices is undefined.

In [2]:
# let's implement Bellman Ford algorithm
# our function with take a graph and a source vertex and return the shortest path from the source to all other vertices
# our graph will be stored as dictionary of keys being tuples of vertices and values being the weight of the edge
# a list of vertices would be nice but not strictly necessary as we can retrieve the vertices from the dictionary
# we will also return a boolean value indicating if there is a negative cycle in the graph

def bellman_form(edge_dict, source, debug=False):
    # first let's extract all vertices from edge_dict, each vertex will appear at least once in the dictionary
    vertices = set()
    for edge in edge_dict:
        vertices.add(edge[0])
        vertices.add(edge[1])

    # here we could check if sources actually is in the graph and early return
    if source not in vertices:
        print(f"ooops! {source} is not in the graph")
        return None

    # first we need to initialize the distance to all vertices to be infinity except the source which is 0
    distance = {}
    for vertex in vertices:
        distance[vertex] = float('inf')
    distance[source] = 0
    # we also need to initialize the predecessor of all vertices to be None
    # we will use this to build the shortest path from the source to any vertex
    predecessor = {}
    for vertex in vertices:
        predecessor[vertex] = None
        # no need to set the predecessor of the source to None as it is already None
    # we need to iterate over all the edges in the graph and update the distance and predecessor of each vertex
    for it in range(len(vertices) - 1): # maximum hops to reach any vertex is the number of vertices - 1
        if debug:
            print(f"iteration {it}")
        for (s,t), w in edge_dict.items():
            if distance[t] > distance[s] + w:
                # means we found a shorter path to t through s
                distance[t] = distance[s] + w
                predecessor[t] = s
    # now we need to check for negative cycles # the end
    for (s,t), w in edge_dict.items():
        if distance[t] > distance[s] + w:
            # means we found a shorter path to t through s
            print(f"ooops! negative cycle detected in the graph! {t} can be reached from {s} with a total weight of {distance[s] + w} which is less than {distance[t]}")
            return (distance, predecessor, True)
    return (distance, predecessor, False)

In [3]:
# so let's build a dictionary to represent the graph
# with vertices a,b,c,d,e
g_dict = {
    ('a', 'b'): 6,
    ('a', 'c'): 7,
    ('a', 'd'): 3,
    ('c', 'b'): 5,
    ('c', 'd'): 4,
    ('d', 'b'): -10,
    ('d', 'e'): 8,
    ('b', 'e'): -2
}
# now let's set source to a and call our function
source = 'a'
result = bellman_form(g_dict, source)
print(result)

({'d': 3, 'b': -7, 'a': 0, 'c': 7, 'e': -9}, {'d': 'a', 'b': 'd', 'a': None, 'c': 'a', 'e': 'b'}, False)


In [4]:
# TODO add print path function
def print_path(s,t, result, debug=False):
    distance, predecessor, negative_cycle = result
    if negative_cycle:
        print(f"ooops! negative cycle detected in the graph! {t} can be reached from {s} with a total weight of {distance[s] + w} which is less than {distance[t]}")
        return None
    path = []
    while t != s:
        path.append(t)
        t = predecessor[t]
    path.append(s)
    path.reverse()
    if debug:
        # print costs for each step
        for step in path:
            print(f"step: {step}, cost: {distance[step]}")
    return path

# let's test our print_path function
print(print_path('a', 'e', result))

['a', 'd', 'b', 'e']


In [5]:
# print with debug
print(print_path('a', 'e', result, debug=True))

step: a, cost: 0
step: d, cost: 3
step: b, cost: -7
step: e, cost: -9
['a', 'd', 'b', 'e']


In [6]:
# let's convert Adjacenty list into edge list
Graph = {
    "SP": {"AR": 2, "O1": 5},
    "AR": {"O1": -2, "NC": 4},
    "O1": {"NC": 2, "O2": 3},
    "O2": {"LCZ": -1},
    "NC": {"O2": 2, "LCZ": 3},
    "LCZ": {}
}

edge_list = {}
for vertex in Graph:
    for neighbor, weight in Graph[vertex].items():
        edge_list[(vertex, neighbor)] = weight

print(*edge_list.items(), sep="\n")

(('SP', 'AR'), 2)
(('SP', 'O1'), 5)
(('AR', 'O1'), -2)
(('AR', 'NC'), 4)
(('O1', 'NC'), 2)
(('O1', 'O2'), 3)
(('O2', 'LCZ'), -1)
(('NC', 'O2'), 2)
(('NC', 'LCZ'), 3)


In [7]:
# let's find our shortest path from SP to LCZ
result = bellman_form(edge_list, "SP")
print(print_path("SP", "LCZ", result, debug=True))

step: SP, cost: 0
step: AR, cost: 2
step: O1, cost: 0
step: O2, cost: 3
step: LCZ, cost: 2
['SP', 'AR', 'O1', 'O2', 'LCZ']


## Converting edge list to adjacency matrix

In [8]:
# let's convert edge list to adjacency matrix
# first let's get the vertices
vertices = set()
for edge in edge_list:
    vertices.add(edge[0])
    vertices.add(edge[1])

# let's initialize the adjacency matrix
adj_matrix = {}
for vertex in vertices:
    adj_matrix[vertex] = {}
    for neighbor in vertices:
        adj_matrix[vertex][neighbor] = float('inf')
    # initialize the diagonal to 0
    adj_matrix[vertex][vertex] = 0

# let's fill the adjacency matrix
for (s,t), w in edge_list.items():
    adj_matrix[s][t] = w

# let's print the adjacency matrix
for vertex in vertices:
    print(f"{vertex} -> {adj_matrix[vertex]}")
    

SP -> {'SP': 0, 'LCZ': inf, 'NC': inf, 'O1': 5, 'O2': inf, 'AR': 2}
LCZ -> {'SP': inf, 'LCZ': 0, 'NC': inf, 'O1': inf, 'O2': inf, 'AR': inf}
NC -> {'SP': inf, 'LCZ': 3, 'NC': 0, 'O1': inf, 'O2': 2, 'AR': inf}
O1 -> {'SP': inf, 'LCZ': inf, 'NC': 2, 'O1': 0, 'O2': 3, 'AR': inf}
O2 -> {'SP': inf, 'LCZ': -1, 'NC': inf, 'O1': inf, 'O2': 0, 'AR': inf}
AR -> {'SP': inf, 'LCZ': inf, 'NC': 4, 'O1': -2, 'O2': inf, 'AR': 0}


In [9]:
# let's convert our adj_matrix to pandas dataframe
import pandas as pd
df = pd.DataFrame(adj_matrix)

In [10]:
df # this matrix approach would be more suitable for dense graphs
# and also for graphs with weighted edges

,SP,LCZ,NC,O1,O2,AR
SP,0.0,inf,inf,inf,inf,inf
LCZ,inf,0.0,3.0,inf,-1.0,inf
NC,inf,inf,0.0,2.0,inf,4.0
O1,5.0,inf,inf,0.0,inf,-2.0
O2,inf,inf,2.0,3.0,0.0,inf
AR,2.0,inf,inf,inf,inf,0.0


In [11]:
# actually a transposition would be more natural because we want
# each row to represent the source vertex and each column to represent the target vertex
df = df.T
df

,SP,LCZ,NC,O1,O2,AR
SP,0.0,inf,inf,5.0,inf,2.0
LCZ,inf,0.0,inf,inf,inf,inf
NC,inf,3.0,0.0,inf,2.0,inf
O1,inf,inf,2.0,0.0,3.0,inf
O2,inf,-1.0,inf,inf,0.0,inf
AR,inf,inf,4.0,-2.0,inf,0.0


In [12]:
# it would not change the result but we can rearrange the rows and columns
# in the following order
order = ["SP", "AR", "O1", "O2", "NC", "LCZ"]
df = df.reindex(order, axis=0)
df = df.reindex(order, axis=1)
df

,SP,AR,O1,O2,NC,LCZ
SP,0.0,2.0,5.0,inf,inf,inf
AR,inf,0.0,-2.0,inf,4.0,inf
O1,inf,inf,0.0,3.0,2.0,inf
O2,inf,inf,inf,0.0,inf,-1.0
NC,inf,inf,inf,2.0,0.0,3.0
LCZ,inf,inf,inf,inf,inf,0.0


## Brute force approach to solving the problem

We could simply check all paths shorter than `|V| - 1` edges and fine the minimal.

Let's assume we have a full/dense graph.
Then we would need to check V-1 edges then V-2 edges then V-3 edges and so on. So complexity for brute force would be O(V!) which is not efficient to say the least... one of the worst possible complexities.

## Alternative to Bellman-Ford for single source shortest path

Dijkstra's algorithm is a more efficient alternative to Bellman-Ford for single source shortest path. It is faster and has a complexity of O(E + VlogV) where E is the number of edges and V is the number of vertices. Again that requires a priority queue to be implemented. Also edge weights must be non-negative.


### Thorup's algorithm

https://dl.acm.org/doi/10.1145/316542.316548

Thorup's algorithm is a more efficient alternative to Bellman-Ford for single source shortest path. It offers the possibility of linear time complexity O(E)